In [11]:
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    n=3,
    messages=[
        {"role": "user", "content": "What is the capital of South Korea?"}
    ]
)

response

ChatCompletion(id='chatcmpl-CFhtTBF1khbVnj2oQ9jQbvN7w5o7J', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)), Choice(finish_reason='stop', index=1, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)), Choice(finish_reason='stop', index=2, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757859683, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=24, prompt_tokens=15, total_tokens=39, completion_tokens_detai

In [12]:
message = response.choices[0].message.content
message

'The capital of South Korea is Seoul.'